# Prepare data from raw scoring results

In [1]:
import pandas as pd
import bioframe as bf
import numpy as np
import os
import sys

In [13]:
DATA_DIR = os.path.dirname(os.getcwd()) + '/data/'

## Read in files 

In [22]:
raw_scoring_results = pd.read_csv(DATA_DIR + 'scoringExamples.tsv', sep='\t')
flipped_scoring_results = pd.read_csv(DATA_DIR + 'scoringExamples_flipped.tsv', sep='\t') 
flipped_scaled_scoring_results = pd.read_csv(DATA_DIR + 'scoringExamples_scaled.tsv', sep='\t')

/Users/lgunsalus/opt/anaconda3/envs/scoring/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [23]:
raw_scoring_results.head()

,chrom,start,end,var_start,var_end,ref,alt,mse,spearman,pearson,...,triangle_spearmanr,triangle_pearsonr,loops,TADs,description,contact_decay_mse,contact_decay_spearmanr,contact_decay_pearsonr,loops_overlap,TADs_overlap
0,chr7,68730553,69779129,69254841,69254941,NaN,NaN,0.002004,0.965683,0.966873,...,0.961940,0.984612,"{'wt': [], 'del': [], 'overlap_loop_in_wt': []...","{'wt': [(392, 0.7149521952031089), (432, 0.205...",random_deletion,0.001119,0.987673,0.989401,0.00,1.0
1,chr20,43529781,44578357,44054069,44054169,NaN,NaN,0.011047,0.981217,0.980891,...,0.985448,0.985902,"{'wt': [(75, 423), (253, 424), (75, 291), (253...","{'wt': [(75, 0.8553585978123123), (240, 0.5196...",random_deletion,0.011141,0.973825,0.964475,0.25,0.5
2,chr14,68810793,69859369,69335081,69335181,NaN,NaN,0.004579,0.990001,0.991952,...,0.991290,0.992468,"{'wt': [(124, 415), (297, 416), (418, 434), (1...","{'wt': [(17, 0.5043706474078453), (113, 0.8153...",random_deletion,0.002087,0.987200,0.986775,0.20,1.0
3,chr11,86622594,87671170,87146882,87146982,NaN,NaN,0.002114,0.996507,0.997073,...,0.997193,0.997305,"{'wt': [(90, 112), (324, 358), (121, 295), (89...","{'wt': [(64, 0.465224714200676), (120, 0.60539...",random_deletion,0.002038,0.993652,0.997388,0.60,1.0
4,chrX,43440115,44488691,43964403,43964503,NaN,NaN,0.000832,0.987883,0.991395,...,0.992785,0.993906,"{'wt': [(27, 119), (73, 118), (28, 72)], 'del'...","{'wt': [(26, 0.7483926295552789), (121, 0.5045...",random_deletion,0.000299,0.962554,0.959670,1.00,1.0


## Examples of processing + normalization applied to raw scores

### Absolute value of eigenvector track

In [24]:
# note: 'DI_mse' is NOT considered
scores_to_consider = ['mse', 'spearman', 'pearson',
       'ssi', 'scc', 'PC_mse', 'PC_spearmanr', 'PC_pearsonr', 
       'DI_spearmanr', 'DI_pearsonr', 'insulation_mse', 'insulation_spearmanr',
       'insulation_pearsonr', 'contact_decay_mse', 'contact_decay_spearmanr',
       'contact_decay_pearsonr', 'triangle_mse', 'triangle_spearmanr',
       'triangle_pearsonr', 'loops_overlap',
       'TADs_overlap']


In [25]:
flipped = raw_scoring_results.copy(deep=True)

for i in scores_to_consider:
    if (i == 'PC_spearmanr') or (i == 'PC_pearsonr'): 
        flipped[i] = abs(flipped[i])

### Flip scores such that a larger value describes a bigger difference

Correlation scores are flipped.

In [26]:
#examples_allTypes_flipped = examples_allTypes.copy(deep=True)
for i in scores_to_consider:
    if ('spearman' in i) or ('pearson' in i) or ('ssi' in i) or ('correlation' in i) or ('scc' in i) or ('overlap' in i) or ('jaccard' in i):
        flipped[i] = 1 - flipped[i]

### Normalize values

In [28]:
# Normalize/scale relative to random_deletion mean
def scaleScores(scores, measure='mean'):
    scores_copy = scores.copy()
    setForNormalizing = flipped[flipped['description'] == 'random_deletion']
    for i in scores_to_consider:
        if measure == 'mean':
            scores_copy[i] = [(x/setForNormalizing[i].mean()) for x in scores[i]]
        elif measure == 'std':
            scores_copy[i] = [(x/setForNormalizing[i].std()) for x in scores[i]]
    return scores_copy

flipped_scaled = scaleScores(flipped) # Takes 2 mins to scale